# 전처리

In [ ]:
import os
import re
import ast
import pandas as pd
import numpy as np
from PIL import Image

import torch
from torchvision import transforms
from datetime import datetime, timedelta

## ID별 DataFrame 분리

In [ ]:
# 경로 및 파일 설정
sensor_items = [
    "mACStatus",
    "mActivity",
    "mAmbience",
    "mBle",
    "mGps",
    "mLight",
    "mScreenStatus",
    "mUsageStats",
    "mWifi",
    "wHr",
    "wLight",
    "wPedo"
]
dataset_path = 'data/ch2025_data_items'
save_root = "data/split_with_id"
os.makedirs(save_root, exist_ok=True)

# 센서별 처리
for sensor in sensor_items:
    file_path = os.path.join(dataset_path, f"ch2025_{sensor}.parquet")

    if not os.path.exists(file_path):
        print(f"❌ File not found: {file_path}")
        continue

    try:
        df = pd.read_parquet(file_path)
    except Exception as e:
        print(f"❌ Failed to load {file_path}: {e}")
        continue

    if "subject_id" not in df.columns:
        print(f"❌ subject_id column missing in {sensor}")
        continue

    # subject_id별로 분리 저장
    for subject_id, df_sub in df.groupby("subject_id"):
        save_dir = os.path.join(save_root, subject_id)
        os.makedirs(save_dir, exist_ok=True)

        save_path = os.path.join(save_dir, f"ch2025_{sensor}.csv")
        df_sub.to_csv(save_path, index=False)

        print(f"✅ Saved: {save_path}")

✅ Saved: data/split_with_id\id01\ch2025_mACStatus.csv
✅ Saved: data/split_with_id\id02\ch2025_mACStatus.csv
✅ Saved: data/split_with_id\id03\ch2025_mACStatus.csv
✅ Saved: data/split_with_id\id04\ch2025_mACStatus.csv
✅ Saved: data/split_with_id\id05\ch2025_mACStatus.csv
✅ Saved: data/split_with_id\id06\ch2025_mACStatus.csv
✅ Saved: data/split_with_id\id07\ch2025_mACStatus.csv
✅ Saved: data/split_with_id\id08\ch2025_mACStatus.csv
✅ Saved: data/split_with_id\id09\ch2025_mACStatus.csv
✅ Saved: data/split_with_id\id10\ch2025_mACStatus.csv
✅ Saved: data/split_with_id\id01\ch2025_mActivity.csv
✅ Saved: data/split_with_id\id02\ch2025_mActivity.csv
✅ Saved: data/split_with_id\id03\ch2025_mActivity.csv
✅ Saved: data/split_with_id\id04\ch2025_mActivity.csv
✅ Saved: data/split_with_id\id05\ch2025_mActivity.csv
✅ Saved: data/split_with_id\id06\ch2025_mActivity.csv
✅ Saved: data/split_with_id\id07\ch2025_mActivity.csv
✅ Saved: data/split_with_id\id08\ch2025_mActivity.csv
✅ Saved: data/split_with_id\

## 결측치 보간
- 데이터에 따라 24시간 기준 다양한 방법 적용

In [ ]:
## 결측치 개수 확인
def check_missing_intervals(df: pd.DataFrame, freq: str = '1d', max_show: int = 10) -> dict:
    df = df.copy()
    df = df.sort_values('timestamp')

    # 기준 단위 결정
    start_time = df['timestamp'].min().floor(freq)
    end_time = df['timestamp'].max().floor(freq)
    full_range = pd.date_range(start=start_time, end=end_time, freq=freq)

    # 실제 존재한 시간 단위 추출
    actual_units = df['timestamp'].dt.floor(freq).unique()
    missing_units = sorted(set(full_range) - set(pd.to_datetime(actual_units)))

    # 최대 연속 누락 계산
    max_gap = 0
    current_gap = 1
    for i in range(1, len(missing_units)):
        if (missing_units[i] - missing_units[i - 1]) == pd.Timedelta(freq):
            current_gap += 1
        else:
            max_gap = max(max_gap, current_gap)
            current_gap = 1
    max_gap = max(max_gap, current_gap) if missing_units else 0

    # 결과 반환
    result = {
        'frequency': freq,
        'start_time': start_time,
        'end_time': end_time,
        'total_intervals': len(full_range),
        'existing_intervals': len(actual_units),
        'missing_intervals': len(missing_units),
        'max_consecutive_missing': max_gap,
        'missing': bool(missing_units)
    }

    return result
## 평균값 보간
def fill_missing_by_time_mean(df: pd.DataFrame, value_col: str, freq: str = '1min') -> pd.DataFrame:
    df = df.copy()
    df = df.set_index('timestamp').sort_index()

    # 전체 타임라인 생성
    full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq=freq)
    df_full = df.reindex(full_range)
    df_full.index.name = 'timestamp'

    # 시간 (HH:MM) 단위 key 만들기
    df_full['hhmm'] = df_full.index.strftime('%H:%M')

    # 기준 시점별 평균값 계산 (결측 제외)
    time_means = df_full.groupby('hhmm')[value_col].mean()

    # 결측값 채우기: hhmm 기준 평균으로
    df_full[value_col] = df_full.apply(
        lambda row: time_means[row['hhmm']] if pd.isna(row[value_col]) else row[value_col],
        axis=1
    )

    # hhmm 컬럼 제거
    df_full = df_full.drop(columns=['hhmm'])

    return df_full.reset_index()
## 최빈값 보간
def fill_missing_by_time_mode(df: pd.DataFrame, value_col: str, freq: str = '1min') -> pd.DataFrame:
    df = df.copy()
    df = df.set_index('timestamp').sort_index()

    # 전체 타임라인 생성
    full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq=freq)
    df_full = df.reindex(full_range)
    df_full.index.name = 'timestamp'

    # 시간 (HH:MM) 단위 key 만들기
    df_full['hhmm'] = df_full.index.strftime('%H:%M')

    # 기준 시점별 최빈값 계산 (결측 제외)
    mode_map = df_full.groupby('hhmm')[value_col].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)

    # 결측값 채우기: hhmm 기준 최빈값으로
    df_full[value_col] = df_full.apply(
        lambda row: mode_map[row['hhmm']] if pd.isna(row[value_col]) else row[value_col],
        axis=1
    )

    df_full = df_full.drop(columns=['hhmm'])
    return df_full.reset_index()

### mACStatus

In [ ]:
# ✅ 반복 처리
for i in range(1, 11):
    subject_id = f"id{i:02d}"
    file_path = f"data/split_with_id/{subject_id}/ch2025_mACStatus.csv"

    if not os.path.exists(file_path):
        print(f"❌ {subject_id}: 파일 없음")
        continue

    # 원본 로딩
    df = pd.read_csv(file_path)
    df['timestamp'] = pd.to_datetime(df['timestamp']).dt.floor('min')

    # 동적 변수 이름으로 저장
    globals()[f"df_mCharging_{i}"] = df

    # 결측 체크
    print(f"\n📌 {subject_id} 결측 체크")
    print("1일:", check_missing_intervals(df, freq='1d'))
    print("1시간:", check_missing_intervals(df, freq='1h'))
    print("1분:", check_missing_intervals(df, freq='1min'))

    # 최빈값 보간
    df_filled = fill_missing_by_time_mode(df, value_col='m_charging', freq='1min')
    df_filled = df_filled.drop(columns=['subject_id'])
    globals()[f"df_mCharging_{i}_filled"] = df_filled

    print("보간 후:")
    print(check_missing_intervals(df_filled, freq='1min'))

    df_filled['m_charging'] = df_filled['m_charging'].astype(np.uint16)
    globals()[f"df_mCharging_{i}_filled"] = df_filled


📌 id01 결측 체크
1일: {'frequency': '1d', 'start_time': Timestamp('2024-06-26 00:00:00'), 'end_time': Timestamp('2024-09-14 00:00:00'), 'total_intervals': 81, 'existing_intervals': 68, 'missing_intervals': 13, 'max_consecutive_missing': 2, 'missing': True}
1시간: {'frequency': '1h', 'start_time': Timestamp('2024-06-26 12:00:00'), 'end_time': Timestamp('2024-09-14 23:00:00'), 'total_intervals': 1932, 'existing_intervals': 1620, 'missing_intervals': 312, 'max_consecutive_missing': 48, 'missing': True}
1분: {'frequency': '1min', 'start_time': Timestamp('2024-06-26 12:03:00'), 'end_time': Timestamp('2024-09-14 23:59:00'), 'total_intervals': 115917, 'existing_intervals': 94266, 'missing_intervals': 21651, 'max_consecutive_missing': 2880, 'missing': True}
보간 후:
{'frequency': '1min', 'start_time': Timestamp('2024-06-26 12:03:00'), 'end_time': Timestamp('2024-09-14 23:59:00'), 'total_intervals': 115917, 'existing_intervals': 115917, 'missing_intervals': 0, 'max_consecutive_missing': 0, 'missing': Fal

In [ ]:
df_mCharging_10_filled.head()

,timestamp,m_charging
0,2024-07-06 00:00:00,1
1,2024-07-06 00:01:00,1
2,2024-07-06 00:02:00,1
3,2024-07-06 00:03:00,1
4,2024-07-06 00:04:00,1


### mActivity

In [ ]:
for i in range(1, 11):
    subject_id = f"id{i:02d}"
    file_path = f"data/split_with_id/{subject_id}/ch2025_mActivity.csv"

    if not os.path.exists(file_path):
        print(f"❌ {subject_id}: 파일 없음")
        continue

    try:
        df = pd.read_csv(file_path)
        df['timestamp'] = pd.to_datetime(df['timestamp']).dt.floor('min')
        globals()[f"df_mActivity_{i}"] = df
    except Exception as e:
        print(f"❌ {subject_id}: 로딩 실패 → {e}")
        continue

    print(f"\n📌 {subject_id} - mActivity 결측 체크")
    print("1일:", check_missing_intervals(df, freq='1d'))
    print("1시간:", check_missing_intervals(df, freq='1h'))
    print("1분:", check_missing_intervals(df, freq='1min'))

    df_filled = fill_missing_by_time_mode(df, value_col='m_activity', freq='1min')
    df_filled = df_filled.drop(columns=['subject_id'])
    globals()[f"df_mActivity_{i}_filled"] = df_filled

    print("보간 후:")
    print(check_missing_intervals(df_filled, freq='1min'))

    df_filled['m_activity'] = (df_filled['m_activity']).astype(np.uint16)
    globals()[f"df_mActivity_{i}_filled"] = df_filled


📌 id01 - mActivity 결측 체크
1일: {'frequency': '1d', 'start_time': Timestamp('2024-06-26 00:00:00'), 'end_time': Timestamp('2024-09-14 00:00:00'), 'total_intervals': 81, 'existing_intervals': 68, 'missing_intervals': 13, 'max_consecutive_missing': 2, 'missing': True}
1시간: {'frequency': '1h', 'start_time': Timestamp('2024-06-26 12:00:00'), 'end_time': Timestamp('2024-09-14 23:00:00'), 'total_intervals': 1932, 'existing_intervals': 1620, 'missing_intervals': 312, 'max_consecutive_missing': 48, 'missing': True}
1분: {'frequency': '1min', 'start_time': Timestamp('2024-06-26 12:03:00'), 'end_time': Timestamp('2024-09-14 23:59:00'), 'total_intervals': 115917, 'existing_intervals': 96976, 'missing_intervals': 18941, 'max_consecutive_missing': 2880, 'missing': True}
보간 후:
{'frequency': '1min', 'start_time': Timestamp('2024-06-26 12:03:00'), 'end_time': Timestamp('2024-09-14 23:59:00'), 'total_intervals': 115917, 'existing_intervals': 115917, 'missing_intervals': 0, 'max_consecutive_missing': 0, 'm

In [ ]:
df_mActivity_7_filled.head()

,timestamp,m_activity
0,2024-06-09 10:51:00,4
1,2024-06-09 10:52:00,3
2,2024-06-09 10:53:00,3
3,2024-06-09 10:54:00,3
4,2024-06-09 10:55:00,3


### mAmbience

<a href="https://research.google.com/audioset/dataset/index.html">Audio-based 527 labels</a> 중에서 대표적인 소리 254개 사용

In [ ]:
## ambience 매핑 리스트
sound_labels = [
    'Silence', 'Sigh', 'Sniff', 'Fowl', 'Trickle, dribble', 'Zipper (clothing)', 'Hum', 'Owl', 'Mains hum',
    'Thump, thud', 'Raindrop', 'Pant', 'Alarm clock', 'Tick', 'Writing', 'Drip',
    'Heart sounds, heartbeat', 'Clock', 'Breathing', 'Humming', 'Purr', 'Biting',
    'Sheep', 'Goat', 'Television', 'Whimper', 'Waterfall', 'Fill (with liquid)', 'Beep, bleep',
    'Plop', 'Electric toothbrush', 'Babbling', 'Sawing', 'Sneeze', 'Noise', 'Synthesizer',
    'Splash, splatter', 'Gasp', 'Beatboxing', 'Whack, thwack', 'Pigeon, dove', 'Pink noise',
    'Drum', 'Boiling', 'Scrape', 'Percussion', 'Buzzer', 'Artillery fire',
    'Whoosh, swoosh, swish', 'Sidetone', 'Slam', 'Goose', 'Squeal', 'Rattle',
    'Computer keyboard', 'Telephone bell ringing', 'Rain', 'Cutlery, silverware',
    'Ratchet, pawl', 'Wind chime', 'Typewriter', 'Telephone dialing, DTMF', 'Hoot',
    'Coin (dropping)', 'Stir', 'Rattle (instrument)', 'Ship', 'Power tool', 'Bouncing',
    'Arrow', 'Jingle bell', 'Wail, moan', 'Groan', 'Hiccup', 'Car', 'Pump (liquid)',
    'Air conditioning', 'Grunt', 'Whistling', 'Subway, metro, underground',
    'Power windows, electric windows', 'Squawk', 'Ice cream truck, ice cream van',
    'Wheeze', 'Sewing machine', 'Rumble', 'Chicken, rooster', 'Pulse', 'Finger snapping',
    'Lullaby', 'Motorcycle', 'Cough', 'Telephone', 'Gush', 'Creak', 'Thunk', 'Ringtone',
    'Crying, sobbing', 'Clang', 'Slap, smack', 'Mantra', 'Sonar', 'Bird', 'Shuffle',
    'Hair dryer', 'Toothbrush', 'Splinter', 'Fart', 'Bus', 'Slosh', 'Basketball bounce',
    'Pour', 'Sound effect', 'Squish', 'Gears', 'Cupboard open or close', 'Ping',
    'Crow', 'Music', 'Crumpling, crinkling', 'Microwave oven', 'Hiss', 'Sink (filling or washing)',
    'Scratch', 'Drawer open or close', 'Rustling leaves', 'Buzz', 'Cap gun', 'Cricket',
    'Bathtub (filling or washing)', 'Electronic tuner', 'Steam', 'Patter', 'Helicopter',
    'Snort', 'Electric shaver, electric razor', 'Rustle', 'Rodents, rats, mice', 'White noise',
    'Tuning fork', 'Chop', 'Tambourine', 'Burst, pop', 'Hands', 'Horse', 'Ocean', 'Fire',
    'Keys jangling', 'Outside, urban or manmade', 'Skateboard', 'Inside, small room',
    'Chink, clink', 'Outside, rural or natural', 'Mechanisms', 'Snake',
    'Traffic noise, roadway noise', 'Crackle', 'Rail transport', 'Liquid',
    'Mechanical fan', 'Wind', 'Spray', 'Whip', 'Insect', 'Motorboat, speedboat',
    'Rowboat, canoe, kayak', 'Pig', 'Crushing', 'Stomach rumble',
    'Dishes, pots, and pans', 'Bell', 'Water tap, faucet', 'Toilet flush', 'Ding',
    'Clip-clop', 'Choir', 'Waves, surf', 'Tap', 'Burping, eructation', 'Wind noise (microphone)',
    'Turkey', 'Camera', 'Wood', 'Environmental noise', 'Water', 'Bee, wasp, etc.', 'Glass',
    'Bicycle', 'Snoring', 'Chime', 'Scissors', 'Sizzle', 'Hammer', 'Printer',
    'Accelerating, revving, vroom', 'Rub', 'Inside, large room or hall', 'Breaking',
    'Chant', 'Crack', 'Frog', 'Aircraft', 'Duck', 'Frying (food)',
    'Vehicle horn, car horn, honking', 'Vacuum cleaner', 'Engine', 'Speech',
    'Sliding door', 'Drill', 'Boat, Water vehicle', 'Child speech, kid speaking',
    'Singing', 'Animal', 'Typing', 'Steam whistle', 'Door', 'Roaring cats (lions, tigers)',
    'Vehicle', 'Wild animals', 'Blender', 'Engine starting', 'Emergency vehicle',
    'Livestock, farm animals, working animals', 'Fire engine, fire truck (siren)', 'Tools',
    'Jet engine', 'Mouse', 'Laughter', 'Cat', 'Whistle', 'Domestic animals, pets',
    'Chewing, mastication', 'Walk, footsteps', 'Cattle, bovinae', 'Child singing', 'Dog',
    'Gunshot, gunfire', 'Crowd', 'Fireworks', 'Alarm', 'Thunderstorm', 'Jackhammer',
    'Machine gun', 'Cheering', 'Chainsaw', 'Firecracker', 'Police car (siren)',
    'Air horn, truck horn', 'Cacophony', 'Siren', 'Ambulance (siren)', 'Explosion',
    'Screaming'
]
print(len(sound_labels))
# 가장 확률 높은 소리 추출 함수
def extract_top_label_ambience(row_str):
    try:
        match = re.search(r"array\(\['([^']+)'", row_str)
        if match:
            return match.group(1)
        return None
    except:
        return None

254


In [ ]:
for i in range(1, 11):
    subject_id = f"id{i:02d}"
    file_path = f"data/split_with_id/{subject_id}/ch2025_mAmbience.csv"

    if not os.path.exists(file_path):
        print(f"❌ {subject_id}: 파일 없음")
        continue

    try:
        df = pd.read_csv(file_path)
        df['timestamp'] = pd.to_datetime(df['timestamp']).dt.floor('min')
    except Exception as e:
        print(f"❌ {subject_id}: 로딩 실패 → {e}")
        continue

    print(f"\n📌 {subject_id} - mAmbience 결측 체크")
    print("1일:", check_missing_intervals(df, freq='1d'))
    print("1시간:", check_missing_intervals(df, freq='1h'))
    print("2분:", check_missing_intervals(df, freq='2min'))

    # 라벨 추출 및 인코딩
    df['top_ambience'] = df['m_ambience'].apply(extract_top_label_ambience)
    df = df.drop(columns=['m_ambience'])

    sound_to_index = {sound: i for i, sound in enumerate(sound_labels)}
    df['top_ambience'] = df['top_ambience'].map(lambda x: sound_to_index.get(x, -1))

    num_removed = (df['top_ambience'] == -1).sum()
    df = df[df['top_ambience'] != -1].copy()
    print(f"제거된 행 수: {num_removed}")

    # 2분 단위 보간
    df_filled = fill_missing_by_time_mode(df, value_col='top_ambience', freq='2min')
    print("보간 후:")
    print(check_missing_intervals(df_filled, freq='2min'))

    # 1분 단위 리샘플링 후 forward fill
    df_filled = df_filled.set_index('timestamp')
    df_resampled = df_filled.resample('1min').ffill().reset_index()
    df_resampled = df_resampled.drop(columns=['subject_id'])
    print("리샘플링 후:")
    print(check_missing_intervals(df_resampled, freq='1min'))

    df_resampled['top_ambience'] = df_resampled['top_ambience'].astype(np.uint16)
    globals()[f"df_mAmbience_{i}_filled"] = df_resampled


📌 id01 - mAmbience 결측 체크
1일: {'frequency': '1d', 'start_time': Timestamp('2024-06-26 00:00:00'), 'end_time': Timestamp('2024-09-14 00:00:00'), 'total_intervals': 81, 'existing_intervals': 68, 'missing_intervals': 13, 'max_consecutive_missing': 2, 'missing': True}
1시간: {'frequency': '1h', 'start_time': Timestamp('2024-06-26 13:00:00'), 'end_time': Timestamp('2024-09-14 23:00:00'), 'total_intervals': 1931, 'existing_intervals': 1619, 'missing_intervals': 312, 'max_consecutive_missing': 48, 'missing': True}
2분: {'frequency': '2min', 'start_time': Timestamp('2024-06-26 13:00:00'), 'end_time': Timestamp('2024-09-14 23:58:00'), 'total_intervals': 57930, 'existing_intervals': 48207, 'missing_intervals': 9723, 'max_consecutive_missing': 1440, 'missing': True}
제거된 행 수: 4
보간 후:
{'frequency': '2min', 'start_time': Timestamp('2024-06-26 13:00:00'), 'end_time': Timestamp('2024-09-14 23:58:00'), 'total_intervals': 57930, 'existing_intervals': 57930, 'missing_intervals': 0, 'max_consecutive_missing'

In [ ]:
df_mAmbience_2_filled

,timestamp,top_ambience
0,2024-07-17 13:00:00,207
1,2024-07-17 13:01:00,207
2,2024-07-17 13:02:00,207
3,2024-07-17 13:03:00,207
4,2024-07-17 13:04:00,207
...,...,...
130254,2024-10-15 23:54:00,150
130255,2024-10-15 23:55:00,150
130256,2024-10-15 23:56:00,207
130257,2024-10-15 23:57:00,207


### mBle

In [ ]:
for i in range(1, 11):
    subject_id = f"id{i:02d}"
    file_path = f"data/split_with_id/{subject_id}/ch2025_mBle.csv"

    if not os.path.exists(file_path):
        print(f"❌ {subject_id}: 파일 없음")
        continue

    try:
        df = pd.read_csv(file_path)
        df['timestamp'] = pd.to_datetime(df['timestamp']).dt.floor('min')
    except Exception as e:
        print(f"❌ {subject_id}: 로딩 실패 → {e}")
        continue

    print(f"\n📌 {subject_id} - mBle 결측 체크")
    print("1일:", check_missing_intervals(df, freq='1d'))
    print("1시간:", check_missing_intervals(df, freq='1h'))
    print("10분:", check_missing_intervals(df, freq='10min'))

    # 10분 단위 해당 구간에 값 존재 여부 → 1 or 0
    df = df.set_index('timestamp')
    df_filled = df['m_ble'].resample('10min').apply(lambda x: 1 if not x.isna().all() else 0)
    df_filled = df_filled.reset_index().rename(columns={'m_ble': 'm_ble'})

    print("보간 후:")
    print(check_missing_intervals(df_filled, freq='10min'))

    # 1분 단위 리샘플링 및 forward fill
    df_filled = df_filled.set_index('timestamp')
    df_resampled = df_filled.resample('1min').ffill().reset_index()

    print("리샘플링 후:")
    print(check_missing_intervals(df_resampled, freq='1min'))

    df_resampled['m_ble'] = df_resampled['m_ble'].astype(np.uint16)
    globals()[f"df_mBle_{i}_filled"] = df_resampled


📌 id01 - mBle 결측 체크
1일: {'frequency': '1d', 'start_time': Timestamp('2024-06-26 00:00:00'), 'end_time': Timestamp('2024-09-14 00:00:00'), 'total_intervals': 81, 'existing_intervals': 68, 'missing_intervals': 13, 'max_consecutive_missing': 2, 'missing': True}
1시간: {'frequency': '1h', 'start_time': Timestamp('2024-06-26 12:00:00'), 'end_time': Timestamp('2024-09-14 21:00:00'), 'total_intervals': 1930, 'existing_intervals': 1137, 'missing_intervals': 793, 'max_consecutive_missing': 55, 'missing': True}
10분: {'frequency': '10min', 'start_time': Timestamp('2024-06-26 12:10:00'), 'end_time': Timestamp('2024-09-14 21:40:00'), 'total_intervals': 11578, 'existing_intervals': 3288, 'missing_intervals': 8290, 'max_consecutive_missing': 337, 'missing': True}
보간 후:
{'frequency': '10min', 'start_time': Timestamp('2024-06-26 12:10:00'), 'end_time': Timestamp('2024-09-14 21:40:00'), 'total_intervals': 11578, 'existing_intervals': 11578, 'missing_intervals': 0, 'max_consecutive_missing': 0, 'missing':

In [ ]:
df_mBle_10_filled.head()

,timestamp,m_ble
0,2024-07-06 00:00:00,1
1,2024-07-06 00:01:00,1
2,2024-07-06 00:02:00,1
3,2024-07-06 00:03:00,1
4,2024-07-06 00:04:00,1


### mGps

In [ ]:
def fix_and_extract_speed(gps_str):
    try:
        if pd.isna(gps_str):
            return []
        # 중괄호 사이 공백을 쉼표로 바꾸기 (예: } { → }, {)
        fixed_str = re.sub(r'\}\s*\{', '}, {', gps_str.strip())

        # 리스트로 감싸지지 않은 경우도 대비
        if not fixed_str.startswith('['):
            fixed_str = f'[{fixed_str}]'

        # 안전하게 파싱
        gps_data = ast.literal_eval(fixed_str)
        return [item['speed'] for item in gps_data]
    except Exception as e:
        print("❌ 오류 발생:", e)
        print("문제 데이터:", gps_str)
        return []

In [ ]:
for i in range(1, 11):
    subject_id = f"id{i:02d}"
    file_path = f"data/split_with_id/{subject_id}/ch2025_mGps.csv"

    if not os.path.exists(file_path):
        print(f"❌ {subject_id}: 파일 없음")
        continue

    try:
        df = pd.read_csv(file_path)
        df['timestamp'] = pd.to_datetime(df['timestamp']).dt.floor('min')
    except Exception as e:
        print(f"❌ {subject_id}: 로딩 실패 → {e}")
        continue

    print(f"\n📌 {subject_id} - mGps 결측 체크")
    print("1일:", check_missing_intervals(df, freq='1d'))
    print("1시간:", check_missing_intervals(df, freq='1h'))
    print("1분:", check_missing_intervals(df, freq='1min'))

    # speed_list 추출 후 평균 속도 계산
    df['speed_list'] = df['m_gps'].apply(fix_and_extract_speed)
    df['m_speed'] = (
        df['speed_list']
        .apply(lambda speeds: sum(speeds) / len(speeds) if speeds else 0)
        .apply(lambda x: round(x * 100))
    )
    df = df.drop(columns=['subject_id', 'm_gps', 'speed_list'])

    # 선형 보간
    df = df.set_index('timestamp')
    df_resampled = df.resample('1min').mean()
    df_filled = df_resampled.interpolate(method='linear').reset_index()

    print("보간 후:")
    print(check_missing_intervals(df_filled, freq='1min'))

    # 로그 스케일 정규화 → 0~1
    max_speed = 8386.0
    df_filled['m_speed'] = (
        np.log1p(df_filled['m_speed']) / np.log1p(max_speed)
    ).clip(0, 1)

    globals()[f"df_mGps_{i}_filled"] = df_filled


📌 id01 - mGps 결측 체크
1일: {'frequency': '1d', 'start_time': Timestamp('2024-06-26 00:00:00'), 'end_time': Timestamp('2024-09-14 00:00:00'), 'total_intervals': 81, 'existing_intervals': 68, 'missing_intervals': 13, 'max_consecutive_missing': 2, 'missing': True}
1시간: {'frequency': '1h', 'start_time': Timestamp('2024-06-26 12:00:00'), 'end_time': Timestamp('2024-09-14 23:00:00'), 'total_intervals': 1932, 'existing_intervals': 1576, 'missing_intervals': 356, 'max_consecutive_missing': 48, 'missing': True}
1분: {'frequency': '1min', 'start_time': Timestamp('2024-06-26 12:03:00'), 'end_time': Timestamp('2024-09-14 23:59:00'), 'total_intervals': 115917, 'existing_intervals': 84634, 'missing_intervals': 31283, 'max_consecutive_missing': 2880, 'missing': True}
보간 후:
{'frequency': '1min', 'start_time': Timestamp('2024-06-26 12:03:00'), 'end_time': Timestamp('2024-09-14 23:59:00'), 'total_intervals': 115917, 'existing_intervals': 115917, 'missing_intervals': 0, 'max_consecutive_missing': 0, 'missin

In [ ]:
df_mGps_3_filled.describe()

,timestamp,m_speed
count,121637,121637.000000
mean,2024-08-28 18:21:00.000000256,0.251260
min,2024-07-17 12:43:00,0.000000
25%,2024-08-07 15:32:00,0.191972
50%,2024-08-28 18:21:00,0.258344
75%,2024-09-18 21:10:00,0.313601
max,2024-10-09 23:59:00,0.926541
std,NaN,0.149335


### mLight

In [ ]:
for i in range(1, 11):
    subject_id = f"id{i:02d}"
    file_path = f"data/split_with_id/{subject_id}/ch2025_mLight.csv"

    if not os.path.exists(file_path):
        print(f"❌ {subject_id}: 파일 없음")
        continue

    try:
        df = pd.read_csv(file_path)
        df['timestamp'] = pd.to_datetime(df['timestamp']).dt.floor('min')
    except Exception as e:
        print(f"❌ {subject_id}: 로딩 실패 → {e}")
        continue

    print(f"\n📌 {subject_id} - mLight 결측 체크")
    print("1일:", check_missing_intervals(df, freq='1d'))
    print("1시간:", check_missing_intervals(df, freq='1h'))
    print("10분:", check_missing_intervals(df, freq='10min'))

    # 평균 보간
    df_filled = fill_missing_by_time_mean(df, value_col='m_light', freq='10min')
    df_filled = df_filled.drop(columns=['subject_id'])

    print("보간 후:")
    print(check_missing_intervals(df_filled, freq='10min'))

    # 1분 단위 리샘플링 및 선형 보간
    df_filled = df_filled.set_index('timestamp')
    df_resampled = df_filled.resample('1min').interpolate(method='linear').reset_index()

    print("리샘플링 후:")
    print(check_missing_intervals(df_resampled, freq='1min'))
    # globals()[f"df_mLight_{i}_filled"] = df_resampled

    # 로그 정규화 (최대 10000 Lux 기준, 초과 시 클리핑 효과)
    max_lux = 10000
    df_resampled['m_light'] = (
        np.log1p(df_resampled['m_light']) / np.log1p(max_lux)
    ).clip(0, 1)
    globals()[f"df_mLight_{i}_filled"] = df_resampled


📌 id01 - mLight 결측 체크
1일: {'frequency': '1d', 'start_time': Timestamp('2024-06-26 00:00:00'), 'end_time': Timestamp('2024-09-14 00:00:00'), 'total_intervals': 81, 'existing_intervals': 68, 'missing_intervals': 13, 'max_consecutive_missing': 2, 'missing': True}
1시간: {'frequency': '1h', 'start_time': Timestamp('2024-06-26 12:00:00'), 'end_time': Timestamp('2024-09-14 23:00:00'), 'total_intervals': 1932, 'existing_intervals': 1620, 'missing_intervals': 312, 'max_consecutive_missing': 48, 'missing': True}
10분: {'frequency': '10min', 'start_time': Timestamp('2024-06-26 12:00:00'), 'end_time': Timestamp('2024-09-14 23:50:00'), 'total_intervals': 11592, 'existing_intervals': 9715, 'missing_intervals': 1877, 'max_consecutive_missing': 288, 'missing': True}
보간 후:
{'frequency': '10min', 'start_time': Timestamp('2024-06-26 12:00:00'), 'end_time': Timestamp('2024-09-14 23:40:00'), 'total_intervals': 11591, 'existing_intervals': 11591, 'missing_intervals': 0, 'max_consecutive_missing': 0, 'missing

In [ ]:
df_mLight_6_filled.describe()

,timestamp,m_light
count,118731,118731.000000
mean,2024-07-14 18:22:00.000000256,0.444844
min,2024-06-03 12:57:00,0.000000
25%,2024-06-24 03:39:30,0.392715
50%,2024-07-14 18:22:00,0.497875
75%,2024-08-04 09:04:30,0.583669
max,2024-08-24 23:47:00,1.000000
std,NaN,0.202189


### mScreenStatus

In [ ]:
for i in range(1, 11):
    subject_id = f"id{i:02d}"
    file_path = f"data/split_with_id/{subject_id}/ch2025_mScreenStatus.csv"

    if not os.path.exists(file_path):
        print(f"❌ {subject_id}: 파일 없음")
        continue

    try:
        df = pd.read_csv(file_path)
        df['timestamp'] = pd.to_datetime(df['timestamp']).dt.floor('min')
    except Exception as e:
        print(f"❌ {subject_id}: 로딩 실패 → {e}")
        continue

    print(f"\n📌 {subject_id} - mScreenStatus 결측 체크")
    print("1일:", check_missing_intervals(df, freq='1d'))
    print("1시간:", check_missing_intervals(df, freq='1h'))
    print("1분:", check_missing_intervals(df, freq='1min'))

    # 최빈값 보간
    df_filled = fill_missing_by_time_mode(df, value_col='m_screen_use', freq='1min')
    df_filled = df_filled.drop(columns=['subject_id'])

    print("보간 후:")
    print(check_missing_intervals(df_filled, freq='1min'))

    df_filled['m_screen_use'] = df_filled['m_screen_use'].astype(np.uint16)
    globals()[f"df_mScreenStatus_{i}_filled"] = df_filled

    # # 0/1 → 0/255 변환
    # df_pixel = df_filled.copy()
    # df_pixel['m_screen_use'] = (df_pixel['m_screen_use'] * 255).round().astype(np.uint8)
    # globals()[f"df_mScreenStatus_{i}_pixel"] = df_pixel


📌 id01 - mScreenStatus 결측 체크
1일: {'frequency': '1d', 'start_time': Timestamp('2024-06-26 00:00:00'), 'end_time': Timestamp('2024-09-14 00:00:00'), 'total_intervals': 81, 'existing_intervals': 68, 'missing_intervals': 13, 'max_consecutive_missing': 2, 'missing': True}
1시간: {'frequency': '1h', 'start_time': Timestamp('2024-06-26 12:00:00'), 'end_time': Timestamp('2024-09-14 23:00:00'), 'total_intervals': 1932, 'existing_intervals': 1620, 'missing_intervals': 312, 'max_consecutive_missing': 48, 'missing': True}
1분: {'frequency': '1min', 'start_time': Timestamp('2024-06-26 12:03:00'), 'end_time': Timestamp('2024-09-14 23:59:00'), 'total_intervals': 115917, 'existing_intervals': 94456, 'missing_intervals': 21461, 'max_consecutive_missing': 2880, 'missing': True}
보간 후:
{'frequency': '1min', 'start_time': Timestamp('2024-06-26 12:03:00'), 'end_time': Timestamp('2024-09-14 23:59:00'), 'total_intervals': 115917, 'existing_intervals': 115917, 'missing_intervals': 0, 'max_consecutive_missing': 0

In [ ]:
df_mScreenStatus_10_filled.head()

,timestamp,m_screen_use
0,2024-07-06 00:00:00,1
1,2024-07-06 00:01:00,1
2,2024-07-06 00:02:00,1
3,2024-07-06 00:03:00,1
4,2024-07-06 00:04:00,1


### mUsageStats

In [ ]:
def extract_total_time(stats_str):
    try:
        # 중괄호 사이 공백을 쉼표로 바꾸기 (예: } { → }, {)
        fixed_str = re.sub(r'\}\s*\{', '}, {', stats_str.strip())

        # 리스트로 감싸지지 않은 경우도 대비
        if not fixed_str.startswith('['):
            fixed_str = f'[{fixed_str}]'

        # 문자열을 리스트로 안전하게 변환
        stats_list = ast.literal_eval(fixed_str)
        # "One UI 홈"을 제외한 항목 중 최대 total_time 반환
        filtered = [item['total_time'] for item in stats_list if item['app_name'].strip() != "One UI 홈"]
        return max(filtered) if filtered else 0
    except Exception as e:
        print(f"Error parsing: {stats_str} -> {e}")
        return 0  # 오류 발생 시 0 반환

In [ ]:
for i in range(1, 11):
    subject_id = f"id{i:02d}"
    file_path = f"data/split_with_id/{subject_id}/ch2025_mUsageStats.csv"

    if not os.path.exists(file_path):
        print(f"❌ {subject_id}: 파일 없음")
        continue

    try:
        df = pd.read_csv(file_path)
        df['timestamp'] = pd.to_datetime(df['timestamp']).dt.floor('min')
    except Exception as e:
        print(f"❌ {subject_id}: 로딩 실패 → {e}")
        continue

    print(f"\n📌 {subject_id} - mUsageStats 결측 체크")
    print("1일:", check_missing_intervals(df, freq='1d'))
    print("1시간:", check_missing_intervals(df, freq='1h'))
    print("10분:", check_missing_intervals(df, freq='10min'))

    # 사용 시간 추출 및 정제
    df['total_time'] = df['m_usage_stats'].apply(extract_total_time)
    df = df.drop(columns=['subject_id', 'm_usage_stats'])

    # 밀리초 → 초
    df['total_time'] = df['total_time'] / 1000

    # 10분 단위 존재 여부 보간
    df = df.set_index('timestamp')
    df_filled = df['total_time'].resample('10min').apply(lambda x: x if not x.isna().all() else 0)
    df_filled = df_filled.reset_index().rename(columns={'total_time': 'total_time'})

    print("보간 후:")
    print(check_missing_intervals(df_filled, freq='10min'))

    # 1분 단위로 리샘플링 후 forward fill
    df_filled = df_filled.set_index('timestamp')
    df_resampled = df_filled.resample('1min').ffill().reset_index()

    print("리샘플링 후:")
    print(check_missing_intervals(df_resampled, freq='1min'))

    max_time = 600.266
    df_resampled['total_time'] = (
        np.log1p(df_resampled['total_time']) / np.log1p(max_time)
    ).clip(0, 1)
    globals()[f"df_mUsageStats_{i}_filled"] = df_resampled


📌 id01 - mUsageStats 결측 체크
1일: {'frequency': '1d', 'start_time': Timestamp('2024-06-26 00:00:00'), 'end_time': Timestamp('2024-09-14 00:00:00'), 'total_intervals': 81, 'existing_intervals': 68, 'missing_intervals': 13, 'max_consecutive_missing': 2, 'missing': True}
1시간: {'frequency': '1h', 'start_time': Timestamp('2024-06-26 13:00:00'), 'end_time': Timestamp('2024-09-14 21:00:00'), 'total_intervals': 1929, 'existing_intervals': 1272, 'missing_intervals': 657, 'max_consecutive_missing': 53, 'missing': True}
10분: {'frequency': '10min', 'start_time': Timestamp('2024-06-26 13:00:00'), 'end_time': Timestamp('2024-09-14 21:50:00'), 'total_intervals': 11574, 'existing_intervals': 5534, 'missing_intervals': 6040, 'max_consecutive_missing': 318, 'missing': True}
보간 후:
{'frequency': '10min', 'start_time': Timestamp('2024-06-26 13:00:00'), 'end_time': Timestamp('2024-09-14 21:50:00'), 'total_intervals': 11574, 'existing_intervals': 11574, 'missing_intervals': 0, 'max_consecutive_missing': 0, 'mi

In [ ]:
df_mUsageStats_3_filled.describe()

,timestamp,total_time
count,121571,121571.000000
mean,2024-08-28 18:04:59.999999488,0.277636
min,2024-07-17 13:00:00,0.000000
25%,2024-08-07 15:32:30,0.000000
50%,2024-08-28 18:05:00,0.000000
75%,2024-09-18 20:37:30,0.712368
max,2024-10-09 23:10:00,0.999431
std,NaN,0.385857


### mWifi (불필요한 데이터 Skip)

In [ ]:
# df_mWifi_1 = pd.read_csv("data/id01/ch2025_mWifi.csv")
# df_mWifi_1['timestamp'] = pd.to_datetime(df_mWifi_1['timestamp']).dt.floor('min')

# result_1d = check_missing_intervals(df_mWifi_1, freq='1d')
# result_1h = check_missing_intervals(df_mWifi_1, freq='1h')
# result_1m = check_missing_intervals(df_mWifi_1, freq='10min')
# print(result_1d, "\n", result_1h, "\n", result_1m, "\n")

### wHr

In [ ]:
def extract_heart_rate_mean(hr_str):
    try:
        if pd.isna(hr_str):
            return np.nan

        # 문자열 내 줄바꿈 제거 및 공백 기준 분리
        fixed_str = re.sub(r'\s+', ' ', hr_str.strip())  # \n 등 제거 후 공백 하나로 통일
        fixed_str = fixed_str.strip('[]')  # 대괄호 제거
        values = [int(val) for val in fixed_str.split() if val.isdigit()]

        return np.mean(values) if values else np.nan
    except Exception as e:
        print("❌ 오류 발생:", e)
        print("문제 데이터:", hr_str)
        return np.nan

In [ ]:
for i in range(1, 11):
    subject_id = f"id{i:02d}"
    file_path = f"data/split_with_id/{subject_id}/ch2025_wHr.csv"

    if not os.path.exists(file_path):
        print(f"❌ {subject_id}: 파일 없음")
        continue

    try:
        df = pd.read_csv(file_path)
        df['timestamp'] = pd.to_datetime(df['timestamp']).dt.floor('min')
    except Exception as e:
        print(f"❌ {subject_id}: 로딩 실패 → {e}")
        continue

    print(f"\n📌 {subject_id} - wHr 결측 체크")
    print("1일:", check_missing_intervals(df, freq='1d'))
    print("1시간:", check_missing_intervals(df, freq='1h'))
    print("1분:", check_missing_intervals(df, freq='1min'))

    # 평균 심박수 계산
    df['heart_rate'] = df['heart_rate'].apply(extract_heart_rate_mean)
    df = df.drop(columns=['subject_id'])

    # 1분 리샘플 + 선형보간 (최대 60분), 나머진 0
    df = df.set_index('timestamp')
    df_resampled = df.resample('1min')
    df_filled = df_resampled.interpolate(method='linear', limit=60).reset_index()
    df_filled['heart_rate'] = df_filled['heart_rate'].fillna(0)

    print("보간 후:")
    print(check_missing_intervals(df_filled, freq='1min'))
    # globals()[f"df_wHr_{i}_filled"] = df_filled

    df_filled['heart_rate'] = df_filled['heart_rate'].round().astype(np.uint16)
    globals()[f"df_wHr_{i}_filled"] = df_filled


📌 id01 - wHr 결측 체크
1일: {'frequency': '1d', 'start_time': Timestamp('2024-06-26 00:00:00'), 'end_time': Timestamp('2024-09-14 00:00:00'), 'total_intervals': 81, 'existing_intervals': 65, 'missing_intervals': 16, 'max_consecutive_missing': 2, 'missing': True}
1시간: {'frequency': '1h', 'start_time': Timestamp('2024-06-26 12:00:00'), 'end_time': Timestamp('2024-09-14 16:00:00'), 'total_intervals': 1925, 'existing_intervals': 758, 'missing_intervals': 1167, 'max_consecutive_missing': 56, 'missing': True}
1분: {'frequency': '1min', 'start_time': Timestamp('2024-06-26 12:23:00'), 'end_time': Timestamp('2024-09-14 16:44:00'), 'total_intervals': 115462, 'existing_intervals': 27166, 'missing_intervals': 88296, 'max_consecutive_missing': 3421, 'missing': True}
보간 후:
{'frequency': '1min', 'start_time': Timestamp('2024-06-26 12:23:00'), 'end_time': Timestamp('2024-09-14 16:44:00'), 'total_intervals': 115462, 'existing_intervals': 115462, 'missing_intervals': 0, 'max_consecutive_missing': 0, 'missing

In [ ]:
df_wHr_1_filled.head()

,timestamp,heart_rate
0,2024-06-26 12:23:00,131
1,2024-06-26 12:24:00,120
2,2024-06-26 12:25:00,120
3,2024-06-26 12:26:00,119
4,2024-06-26 12:27:00,116


### wLight

In [ ]:
for i in range(1, 11):
    subject_id = f"id{i:02d}"
    file_path = f"data/split_with_id/{subject_id}/ch2025_wLight.csv"

    if not os.path.exists(file_path):
        print(f"❌ {subject_id}: 파일 없음")
        continue

    try:
        df = pd.read_csv(file_path)
        df['timestamp'] = pd.to_datetime(df['timestamp']).dt.floor('min')
    except Exception as e:
        print(f"❌ {subject_id}: 로딩 실패 → {e}")
        continue

    print(f"\n📌 {subject_id} - wLight 결측 체크")
    print("1일:", check_missing_intervals(df, freq='1d'))
    print("1시간:", check_missing_intervals(df, freq='1h'))
    print("1분:", check_missing_intervals(df, freq='1min'))

    # 컬럼 정제 및 보간
    df = df.drop(columns=['subject_id'])
    df = df.set_index('timestamp')
    df_resampled = df.resample('1min')
    df_filled = df_resampled.interpolate(method='linear', limit=60).reset_index()
    df_filled['w_light'] = df_filled['w_light'].fillna(0)

    print("보간 후:")
    print(check_missing_intervals(df_filled, freq='1min'))

    # 1차 로그 스케일링
    max_lux_raw = 10000
    df_filled['w_light'] = (np.log1p(df_filled['w_light']) / np.log1p(max_lux_raw)).clip(0, 1)

    print("로그 스케일링 후:")
    print(check_missing_intervals(df_filled, freq='1min'))
    globals()[f"df_wLight_{i}_filled"] = df_filled


📌 id01 - wLight 결측 체크
1일: {'frequency': '1d', 'start_time': Timestamp('2024-06-26 00:00:00'), 'end_time': Timestamp('2024-09-14 00:00:00'), 'total_intervals': 81, 'existing_intervals': 67, 'missing_intervals': 14, 'max_consecutive_missing': 2, 'missing': True}
1시간: {'frequency': '1h', 'start_time': Timestamp('2024-06-26 12:00:00'), 'end_time': Timestamp('2024-09-14 23:00:00'), 'total_intervals': 1932, 'existing_intervals': 1535, 'missing_intervals': 397, 'max_consecutive_missing': 48, 'missing': True}
1분: {'frequency': '1min', 'start_time': Timestamp('2024-06-26 12:17:00'), 'end_time': Timestamp('2024-09-14 23:59:00'), 'total_intervals': 115903, 'existing_intervals': 42593, 'missing_intervals': 73310, 'max_consecutive_missing': 2887, 'missing': True}
보간 후:
{'frequency': '1min', 'start_time': Timestamp('2024-06-26 12:17:00'), 'end_time': Timestamp('2024-09-14 23:59:00'), 'total_intervals': 115903, 'existing_intervals': 115903, 'missing_intervals': 0, 'max_consecutive_missing': 0, 'miss

In [ ]:
df_wLight_6_filled.describe()

,timestamp,w_light
count,118652,118652.000000
mean,2024-07-14 19:13:29.999999744,0.238558
min,2024-06-03 14:28:00,0.000000
25%,2024-06-24 04:50:45,0.000000
50%,2024-07-14 19:13:30,0.179156
75%,2024-08-04 09:36:15,0.492115
max,2024-08-24 23:59:00,1.000000
std,NaN,0.256124


### wPedo

In [ ]:
for i in range(1, 11):
    subject_id = f"id{i:02d}"
    file_path = f"data/split_with_id/{subject_id}/ch2025_wPedo.csv"

    if not os.path.exists(file_path):
        print(f"❌ {subject_id}: 파일 없음")
        continue

    try:
        df = pd.read_csv(file_path)
        df['timestamp'] = pd.to_datetime(df['timestamp']).dt.floor('min')
    except Exception as e:
        print(f"❌ {subject_id}: 로딩 실패 → {e}")
        continue

    print(f"\n📌 {subject_id} - wPedo 결측 체크")
    print("1일:", check_missing_intervals(df, freq='1d'))
    print("1시간:", check_missing_intervals(df, freq='1h'))
    print("1분:", check_missing_intervals(df, freq='1min'))

    # 컬럼 정제 및 보간
    df = df.drop(columns=['subject_id', 'running_step', 'walking_step'])
    df = df.set_index('timestamp')
    df_resampled = df.resample('1min')
    df_filled = df_resampled.interpolate(method='linear', limit=60).reset_index()

    # 남은 결측값: 최빈값 보간
    for col in ['step', 'step_frequency', 'distance', 'speed', 'burned_calories']:
        df_filled = fill_missing_by_time_mode(df_filled, value_col=col, freq='1min')

    print("보간 후:")
    print(check_missing_intervals(df_filled, freq='1min'))

    SCALING_MAX = {          # “log1p → 0-1”에 사용할 전역 upper-bound
        "step"          : 317.0,
        "step_frequency": 5.3,
        "distance"      : 302.25,
        "speed"         : 5.05,
        "burned_calories": 293.2,
    }

    def log1p_minmax(col, max_val):
        """log1p → 0-1 스케일, 결측은 0으로 유지"""
        return np.where(
            col > 0,
            np.log1p(col) / np.log1p(max_val),   # 0~1 float32
            0.0
        ).astype(np.float32)

    for col, m in SCALING_MAX.items():
        df_filled[col] = log1p_minmax(df_filled[col].values, m)

    globals()[f"df_wPedo_{i}_filled"] = df_filled


📌 id01 - wPedo 결측 체크
1일: {'frequency': '1d', 'start_time': Timestamp('2024-06-26 00:00:00'), 'end_time': Timestamp('2024-09-14 00:00:00'), 'total_intervals': 81, 'existing_intervals': 67, 'missing_intervals': 14, 'max_consecutive_missing': 2, 'missing': True}
1시간: {'frequency': '1h', 'start_time': Timestamp('2024-06-26 12:00:00'), 'end_time': Timestamp('2024-09-14 23:00:00'), 'total_intervals': 1932, 'existing_intervals': 1537, 'missing_intervals': 395, 'max_consecutive_missing': 48, 'missing': True}
1분: {'frequency': '1min', 'start_time': Timestamp('2024-06-26 12:09:00'), 'end_time': Timestamp('2024-09-14 23:59:00'), 'total_intervals': 115911, 'existing_intervals': 86154, 'missing_intervals': 29757, 'max_consecutive_missing': 2880, 'missing': True}
보간 후:
{'frequency': '1min', 'start_time': Timestamp('2024-06-26 12:09:00'), 'end_time': Timestamp('2024-09-14 23:59:00'), 'total_intervals': 115911, 'existing_intervals': 115911, 'missing_intervals': 0, 'max_consecutive_missing': 0, 'missi

In [ ]:
df_wPedo_1_filled.describe()

,timestamp,step,step_frequency,distance,speed,burned_calories
count,115911,115911.000000,115911.000000,115911.000000,115911.000000,115911.000000
mean,2024-08-05 18:04:00.000000512,0.036856,0.015220,0.034382,0.012774,0.007603
min,2024-06-26 12:09:00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2024-07-16 15:06:30,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2024-08-05 18:04:00,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2024-08-25 21:01:30,0.000000,0.000000,0.000000,0.000000,0.000000
max,2024-09-14 23:59:00,0.979149,0.944032,0.982218,0.951994,0.962417
std,NaN,0.145078,0.071391,0.136076,0.060807,0.037927


## DataFrame 변형

In [ ]:
def assign_lifelog_day(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # 기준 시각(06:00) 이전이면 하루 전 날짜를 lifelog로 설정
    df['lifelog_date'] = df['timestamp'].apply(
        lambda x: (x - pd.Timedelta(days=1)).date() if x.time() < pd.to_datetime("06:00").time() else x.date()
    )
    return df

def format_time_label(ts):
    hour = ts.hour
    minute = ts.minute
    return f"{hour:02d}_{minute:02d}"

def restructure_lifelog(df, values):
    df = assign_lifelog_day(df)

    # 시간 단위 컬럼 생성
    df['time_col'] = df['timestamp'].apply(format_time_label)

    # 피벗: lifelog_date를 인덱스로, time_col을 열로 사용
    df_pivot = df.pivot(index='lifelog_date', columns='time_col', values=values)

    # 06:00 ~ 다음날 05:59 순서 정렬
    ordered_cols = [f"{h:02d}_{m:02d}" for h in list(range(6, 24)) + list(range(0, 6)) for m in range(0, 60)]
    ordered_cols = [col for col in ordered_cols if col in df_pivot.columns]

    df_pivot = df_pivot[ordered_cols]

    return df_pivot.reset_index()

def fill_with_mode(col: pd.Series) -> pd.Series:
            """NaN을 컬럼 최빈값(여러 개면 첫 번째)으로 채우고 정수로 변환"""
            if col.isna().all():                 # 전부 NaN이면 0으로 대체(선택)
                mode_val = 0
            else:
                mode_val = col.mode(dropna=True).iloc[0]
            return (
                col.fillna(mode_val)             # NaN → 최빈값
                   .round()                      # 소수점이 있으면 반올림
                   .astype("uint16")               # 필요한 경우 int16/int32
            )

### mACStatus

In [ ]:
for i in range(1, 11):
    try:
        df_pixel = globals()[f"df_mCharging_{i}_filled"]
        df_transformed = restructure_lifelog(df_pixel, 'm_charging')
        df_transformed['lifelog_date'] = pd.to_datetime(df_transformed['lifelog_date'])
        df_transformed['data'] = 'm_charging'

        num_cols = df_transformed.select_dtypes("number").columns
        df_transformed[num_cols] = df_transformed[num_cols].apply(fill_with_mode)

        globals()[f"df_mCharging_{i}_transformed"] = df_transformed
    except KeyError:
        print(f"❌ df_mCharging_{i}_filled 없음")
    except Exception as e:
        print(f"⚠️ df_mCharging_{i}_filled 처리 중 오류: {e}")

In [ ]:
df_mCharging_2_transformed.head()

time_col,lifelog_date,06_00,06_01,06_02,06_03,06_04,06_05,06_06,06_07,06_08,...,05_51,05_52,05_53,05_54,05_55,05_56,05_57,05_58,05_59,data
0,2024-07-17,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,m_charging
1,2024-07-18,1,1,1,1,1,1,0,0,0,...,1,1,1,1,1,1,1,1,1,m_charging
2,2024-07-19,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,m_charging
3,2024-07-20,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,m_charging
4,2024-07-21,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,m_charging


### mActivity

In [ ]:
for i in range(1, 11):
    try:
        df_pixel = globals()[f"df_mActivity_{i}_filled"]
        df_transformed = restructure_lifelog(df_pixel, 'm_activity')
        df_transformed['lifelog_date'] = pd.to_datetime(df_transformed['lifelog_date'])
        df_transformed['data'] = 'm_activity'

        num_cols = df_transformed.select_dtypes("number").columns
        df_transformed[num_cols] = df_transformed[num_cols].apply(fill_with_mode)

        globals()[f"df_mActivity_{i}_transformed"] = df_transformed
    except KeyError:
        print(f"❌ df_mActivity_{i}_filled 없음")
    except Exception as e:
        print(f"⚠️ df_mActivity_{i}_filled 처리 중 오류: {e}")

### mAmbience

In [ ]:
for i in range(1, 11):
    try:
        df_pixel = globals()[f"df_mAmbience_{i}_filled"]
        df_transformed = restructure_lifelog(df_pixel, 'top_ambience')
        df_transformed['lifelog_date'] = pd.to_datetime(df_transformed['lifelog_date'])
        df_transformed['data'] = 'm_ambience'

        num_cols = df_transformed.select_dtypes("number").columns
        df_transformed[num_cols] = df_transformed[num_cols].apply(fill_with_mode)

        globals()[f"df_mAmbience_{i}_transformed"] = df_transformed
    except KeyError:
        print(f"❌ df_mAmbience_{i}_filled 없음")
    except Exception as e:
        print(f"⚠️ df_mAmbience_{i}_filled 처리 중 오류: {e}")

In [ ]:
df_mAmbience_1_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Columns: 1442 entries, lifelog_date to data
dtypes: datetime64[ns](1), object(1), uint16(1440)
memory usage: 229.2+ KB


### mBle

In [ ]:
for i in range(1, 11):
    try:
        df_pixel = globals()[f"df_mBle_{i}_filled"]
        df_transformed = restructure_lifelog(df_pixel, 'm_ble')
        df_transformed['lifelog_date'] = pd.to_datetime(df_transformed['lifelog_date'])
        df_transformed['data'] = 'm_ble'

        num_cols = df_transformed.select_dtypes("number").columns
        df_transformed[num_cols] = df_transformed[num_cols].apply(fill_with_mode)

        globals()[f"df_mBle_{i}_transformed"] = df_transformed
    except KeyError:
        print(f"❌ df_mBle_{i}_filled 없음")
    except Exception as e:
        print(f"⚠️ df_mBle_{i}_filled 처리 중 오류: {e}")

### mGps

In [ ]:
for i in range(1, 11):
    try:
        df_pixel = globals()[f"df_mGps_{i}_filled"]
        df_transformed = restructure_lifelog(df_pixel, 'm_speed')
        df_transformed['lifelog_date'] = pd.to_datetime(df_transformed['lifelog_date'])
        df_transformed['data'] = 'm_speed'

        value_cols = df_transformed.select_dtypes("number").columns
        # ④ 컬럼별 평균으로 NaN 채우기
        for c in value_cols:
            mean_val = df_transformed[c].mean(skipna=True)
            df_transformed[c] = df_transformed[c].fillna(mean_val)

        globals()[f"df_mGps_{i}_transformed"] = df_transformed
    except KeyError:
        print(f"❌ df_mGps_{i}_filled 없음")
    except Exception as e:
        print(f"⚠️ df_mGps_{i}_filled 처리 중 오류: {e}")

### mLight

In [ ]:
for i in range(1, 11):
    try:
        df_pixel = globals()[f"df_mLight_{i}_filled"]
        df_transformed = restructure_lifelog(df_pixel, 'm_light')
        df_transformed['lifelog_date'] = pd.to_datetime(df_transformed['lifelog_date'])
        df_transformed['data'] = 'm_light'

        value_cols = df_transformed.select_dtypes("number").columns
        # ④ 컬럼별 평균으로 NaN 채우기
        for c in value_cols:
            mean_val = df_transformed[c].mean(skipna=True)
            df_transformed[c] = df_transformed[c].fillna(mean_val)

        globals()[f"df_mLight_{i}_transformed"] = df_transformed
    except KeyError:
        print(f"❌ df_mLight_{i}_filled 없음")
    except Exception as e:
        print(f"⚠️ df_mLight_{i}_filled 처리 중 오류: {e}")

### mScreenStatus

In [ ]:
for i in range(1, 11):
    try:
        df_pixel = globals()[f"df_mScreenStatus_{i}_filled"]
        df_transformed = restructure_lifelog(df_pixel, 'm_screen_use')
        df_transformed['lifelog_date'] = pd.to_datetime(df_transformed['lifelog_date'])
        df_transformed['data'] = 'm_screen_use'

        num_cols = df_transformed.select_dtypes("number").columns
        df_transformed[num_cols] = df_transformed[num_cols].apply(fill_with_mode)

        globals()[f"df_mScreenStatus_{i}_transformed"] = df_transformed
    except KeyError:
        print(f"❌ df_mScreenStatus_{i}_filled 없음")
    except Exception as e:
        print(f"⚠️ df_mScreenStatus_{i}_filled 처리 중 오류: {e}")

### mUsageStats

In [ ]:
for i in range(1, 11):
    try:
        df_pixel = globals()[f"df_mUsageStats_{i}_filled"]
        df_transformed = restructure_lifelog(df_pixel, 'total_time')
        df_transformed['lifelog_date'] = pd.to_datetime(df_transformed['lifelog_date'])
        df_transformed['data'] = 'm_usage_stats'

        value_cols = df_transformed.select_dtypes("number").columns
        # ④ 컬럼별 평균으로 NaN 채우기
        for c in value_cols:
            mean_val = df_transformed[c].mean(skipna=True)
            df_transformed[c] = df_transformed[c].fillna(mean_val)

        globals()[f"df_mUsageStats_{i}_transformed"] = df_transformed
    except KeyError:
        print(f"❌ df_mUsageStats_{i}_filled 없음")
    except Exception as e:
        print(f"⚠️ df_mUsageStats_{i}_filled 처리 중 오류: {e}")

### wHr

In [ ]:
for i in range(1, 11):
    try:
        df_pixel = globals()[f"df_wHr_{i}_filled"]
        df_transformed = restructure_lifelog(df_pixel, 'heart_rate')
        df_transformed['lifelog_date'] = pd.to_datetime(df_transformed['lifelog_date'])
        df_transformed['data'] = 'heart_rate'

        value_cols = df_transformed.select_dtypes("number").columns
        # ④ 컬럼별 평균으로 NaN 채우기
        for c in value_cols:
            mean_val = df_transformed[c].mean(skipna=True)
            df_transformed[c] = df_transformed[c].fillna(mean_val)

        globals()[f"df_wHr_{i}_transformed"] = df_transformed
    except KeyError:
        print(f"❌ df_wHr_{i}_filled 없음")
    except Exception as e:
        print(f"⚠️ df_wHr_{i}_filled 처리 중 오류: {e}")

In [ ]:
df_wHr_1_transformed.head()

time_col,lifelog_date,06_00,06_01,06_02,06_03,06_04,06_05,06_06,06_07,06_08,...,05_51,05_52,05_53,05_54,05_55,05_56,05_57,05_58,05_59,data
0,2024-06-26,40.9125,40.2125,40.95,42.6875,42.7,43.3625,43.775,43.35,43.3875,...,78.0,89.0,102.0,105.0,84.0,87.0,89.0,71.0,75.0,heart_rate
1,2024-06-27,72.0000,75.0000,79.00,83.0000,84.0,85.0000,85.000,85.00,96.0000,...,103.0,98.0,99.0,101.0,98.0,100.0,85.0,78.0,79.0,heart_rate
2,2024-06-28,87.0000,91.0000,102.00,94.0000,92.0,99.0000,91.000,92.00,86.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,heart_rate
3,2024-06-29,0.0000,0.0000,0.00,0.0000,0.0,0.0000,0.000,0.00,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,heart_rate
4,2024-06-30,0.0000,0.0000,0.00,0.0000,0.0,0.0000,0.000,0.00,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,heart_rate


### wLight

In [ ]:
for i in range(1, 11):
    try:
        df_pixel = globals()[f"df_wLight_{i}_filled"]
        df_transformed = restructure_lifelog(df_pixel, 'w_light')
        df_transformed['lifelog_date'] = pd.to_datetime(df_transformed['lifelog_date'])
        df_transformed['data'] = 'w_light'

        value_cols = df_transformed.select_dtypes("number").columns
        # ④ 컬럼별 평균으로 NaN 채우기
        for c in value_cols:
            mean_val = df_transformed[c].mean(skipna=True)
            df_transformed[c] = df_transformed[c].fillna(mean_val)

        globals()[f"df_wLight_{i}_transformed"] = df_transformed
    except KeyError:
        print(f"❌ df_wLight_{i}_filled 없음")
    except Exception as e:
        print(f"⚠️ df_wLight_{i}_filled 처리 중 오류: {e}")

### wPedo

In [ ]:
lifelog_columns = ['step', 'step_frequency', 'distance', 'speed', 'burned_calories']

for i in range(1, 11):
    try:
        df_pixel = globals()[f"df_wPedo_{i}_filled"]

        # 결과 리스트
        df_transformed_list = []

        for col in lifelog_columns:
            df_single = restructure_lifelog(df_pixel[['timestamp', col]].copy(), col)
            df_single['lifelog_date'] = pd.to_datetime(df_single['lifelog_date'])
            df_single['data'] = col

            num_cols = df_single.select_dtypes("number").columns
            for c in num_cols:
                    df_single[c] = df_single[c].fillna(df_single[c].mean(skipna=True))
            df_transformed_list.append(df_single)

        # 병합
        df_merged = pd.concat(df_transformed_list, axis=0, ignore_index=True)
        globals()[f"df_wPedo_{i}_merged"] = df_merged

    except KeyError:
        print(f"❌ df_wPedo_{i}_filled 없음")
    except Exception as e:
        print(f"⚠️ df_wPedo_{i}_filled 처리 중 오류: {e}")

In [ ]:
df_wPedo_2_merged[df_wPedo_2_merged['data'] == 'step'].tail()

time_col,lifelog_date,06_00,06_01,06_02,06_03,06_04,06_05,06_06,06_07,06_08,...,05_51,05_52,05_53,05_54,05_55,05_56,05_57,05_58,05_59,data
86,2024-10-11,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00000,0.000000,step
87,2024-10-12,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00000,0.000000,step
88,2024-10-13,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.416153,0.000000,0.0,0.0,0.00000,0.000000,step
89,2024-10-14,0.0,0.0,0.0,0.0,0.0,0.902196,0.848717,0.501622,0.640202,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00000,0.000000,step
90,2024-10-15,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.018999,0.040007,0.0,0.0,0.01404,0.011592,step


## 모든 DataFrame 결합

In [ ]:
sensor_order = [
    'heart_rate', 'burned_calories', 'step', 'step_frequency', 'speed', 'm_speed',
    'distance', 'm_activity', 'm_ambience', 'm_light', 'w_light', 'm_ble',
    'm_screen_use', 'm_usage_stats', 'm_charging'
]
mean_sensors = {
    'heart_rate', 'burned_calories', 'step', 'step_frequency', 'speed', 'm_speed',
    'distance', 'm_light', 'w_light', 'm_usage_stats'
}

In [ ]:
for i in range(1, 11):
    try:
        merged_list = [
            globals()[f"df_mCharging_{i}_transformed"],
            globals()[f"df_mActivity_{i}_transformed"],
            globals()[f"df_mAmbience_{i}_transformed"],
            globals()[f"df_mBle_{i}_transformed"],
            globals()[f"df_mGps_{i}_transformed"],
            globals()[f"df_mLight_{i}_transformed"],
            globals()[f"df_mScreenStatus_{i}_transformed"],
            globals()[f"df_mUsageStats_{i}_transformed"],
            globals()[f"df_wHr_{i}_transformed"],
            globals()[f"df_wLight_{i}_transformed"],
            globals()[f"df_wPedo_{i}_merged"]
        ]

        df_merged = pd.concat(merged_list, axis=0, ignore_index=True)
        df_sorted = df_merged.sort_values(by='lifelog_date').reset_index(drop=True)

        # ✅ 누락된 (lifelog_date, sensor) 보정
        unique_dates = df_sorted['lifelog_date'].unique()
        existing_pairs = set(zip(df_sorted['lifelog_date'], df_sorted['data']))

        missing_rows = []
        for date in unique_dates:
            for sensor in sensor_order:
                if (date, sensor) not in existing_pairs:
                    row = {'lifelog_date': date, 'data': sensor}
                    missing_rows.append(row)

        print(missing_rows if missing_rows else "누락 없음")
        # ✅ 컬럼 정의 및 초기화
        time_columns = [col for col in df_sorted.columns if col not in ['lifelog_date', 'data']]
        df_missing = pd.DataFrame(missing_rows)
        nan_matrix = pd.DataFrame(np.nan, index=df_missing.index, columns=time_columns)
        df_missing = pd.concat([df_missing, nan_matrix], axis=1)
        # ✅ 합치기
        df_augmented = pd.concat([df_sorted, df_missing], ignore_index=True)

        # ✅ 정렬
        df_augmented['data'] = pd.Categorical(df_augmented['data'], categories=sensor_order, ordered=True)
        df_augmented = df_augmented.sort_values(by=['lifelog_date', 'data']).reset_index(drop=True)

        # ✅ 평균 기반 보간 및 타입 변환
        df_filled = df_augmented.copy()
        for col in time_columns:
            # ── ① 센서별 평균과 최빈값 사전 계산 ─────────────────────────────
            mean_map = df_filled.groupby('data', observed=False)[col].mean()
            mode_map = df_filled.groupby('data', observed=False)[col].agg(lambda s: s.mode(dropna=True).iloc[0] if not s.mode().empty else np.nan)
            # NaN인 경우에만 센서별 평균or최빈값으로 대체
            df_filled[col] = df_filled.apply(
                lambda row: (mean_map[row['data']] if (pd.isna(row[col]) and row['data'] in mean_sensors)
                             else mode_map[row['data']]  if (pd.isna(row[col])) else row[col]),
                axis=1
            )

        # for s in sensor_order:
        #     mask = df_filled['data'] == s            # 해당 센서 행만 선택
        #     if s not in mean_sensors:
        #         df_filled.loc[mask, time_columns] = (
        #             df_filled.loc[mask, time_columns]
        #             .round().clip(lower=0).astype('uint16')
        #         )
        #     elif s in mean_sensors:
        #         df_filled.loc[mask, time_columns] = (
        #             df_filled.loc[mask, time_columns]
        #             .astype('float32')
        #         )
        # ✅ 모든 센서-시간 열을 float32로 통일
        time_columns = [c for c in df_filled.columns if c not in ('lifelog_date', 'data')]
        df_filled[time_columns] = df_filled[time_columns].astype('float32')   # 또는 'Float32'

        # ✅ 저장
        globals()[f"df_lifelog_{i}_cleaned"] = df_filled
        print(f"✅ id{i:02d} 처리 완료: shape={df_filled.shape}")

    except KeyError as e:
        print(f"❌ 센서 데이터 없음 (i={i}): {e}")
    except Exception as e:
        print(f"⚠️ 처리 중 오류 (i={i}): {e}")

누락 없음
✅ id01 처리 완료: shape=(1215, 1442)
누락 없음
✅ id02 처리 완료: shape=(1365, 1442)
누락 없음
✅ id03 처리 완료: shape=(1275, 1442)
누락 없음
✅ id04 처리 완료: shape=(1365, 1442)
누락 없음
✅ id05 처리 완료: shape=(1260, 1442)
[{'lifelog_date': Timestamp('2024-06-03 00:00:00'), 'data': 'm_usage_stats'}, {'lifelog_date': Timestamp('2024-06-04 00:00:00'), 'data': 'm_usage_stats'}, {'lifelog_date': Timestamp('2024-06-05 00:00:00'), 'data': 'm_usage_stats'}, {'lifelog_date': Timestamp('2024-06-06 00:00:00'), 'data': 'm_usage_stats'}, {'lifelog_date': Timestamp('2024-06-07 00:00:00'), 'data': 'm_usage_stats'}, {'lifelog_date': Timestamp('2024-06-08 00:00:00'), 'data': 'm_usage_stats'}, {'lifelog_date': Timestamp('2024-06-09 00:00:00'), 'data': 'm_usage_stats'}, {'lifelog_date': Timestamp('2024-06-10 00:00:00'), 'data': 'm_usage_stats'}, {'lifelog_date': Timestamp('2024-06-11 00:00:00'), 'data': 'm_usage_stats'}, {'lifelog_date': Timestamp('2024-06-12 00:00:00'), 'data': 'm_usage_stats'}]
✅ id06 처리 완료: shape=(1245, 1442)
누

In [ ]:
df_lifelog_1_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1215 entries, 0 to 1214
Columns: 1442 entries, lifelog_date to data
dtypes: category(1), datetime64[ns](1), float32(1440)
memory usage: 6.7 MB


In [ ]:
for i in range(1, 11):
    df_name = f"df_lifelog_{i}_cleaned"
    df = globals().get(df_name)

    if df is None:                       # 데이터프레임이 없을 때
        print(f"❌ {df_name} 없음")
        continue

    # 전체 NaN 개수
    total_nan = df.isna().sum().sum()

    # NaN 이 존재하는 컬럼과 그 개수
    nan_cols = df.isna().sum()
    nan_cols = nan_cols[nan_cols > 0]

    if total_nan == 0:
        print(f"✅ {df_name}: NaN 없음")
    else:
        print(f"⚠️ {df_name}: NaN {total_nan}개")
        print("   ▸ NaN 존재 컬럼:", list(nan_cols.index))
        # 필요하면 세부 카운트도 출력
        # print(nan_cols.to_string())

✅ df_lifelog_1_cleaned: NaN 없음
✅ df_lifelog_2_cleaned: NaN 없음
✅ df_lifelog_3_cleaned: NaN 없음
✅ df_lifelog_4_cleaned: NaN 없음
✅ df_lifelog_5_cleaned: NaN 없음
✅ df_lifelog_6_cleaned: NaN 없음
✅ df_lifelog_7_cleaned: NaN 없음
✅ df_lifelog_8_cleaned: NaN 없음
✅ df_lifelog_9_cleaned: NaN 없음
✅ df_lifelog_10_cleaned: NaN 없음


In [ ]:
def expected_dtype(sensor: str):
    """센서 이름에 따라 기대 자료형 반환"""
    return np.float32 if sensor in mean_sensors else np.uint16   # 필요시 uint16

def check_dtypes(df: pd.DataFrame, label: str):
    """df_lifelog_*_cleaned 하나에 대해 센서별 실제 dtype을 확인"""
    time_cols = [c for c in df.columns if c not in ('lifelog_date', 'data')]

    rows = []
    for sensor in sensor_order:
        # 센서별 서브셋 (첫 행만 검사해도 됨 → dtype은 컬럼 단위라 동일)
        sub = df.loc[df['data'] == sensor, time_cols]
        if sub.empty:
            actual = '-'        # 해당 센서가 없는 경우
        else:
            actual = str(sub.dtypes.mode().iloc[0])  # 가장 흔한 dtype
        rows.append({
            'sensor'   : sensor,
            'expected' : str(expected_dtype(sensor)),
            'actual'   : actual,
            'match'    : actual == str(expected_dtype(sensor))
        })

    result = pd.DataFrame(rows)
    print(f"\n📋 {label} dtype 점검")
    print(result.to_string(index=False))

#------------- 실행 -------------#
for i in range(1, 11):
    df_name = f"df_lifelog_{i}_cleaned"
    df = globals().get(df_name)
    if df is None:
        print(f"❌ {df_name} 없음")
    else:
        check_dtypes(df, df_name)


📋 df_lifelog_1_cleaned dtype 점검
         sensor                expected  actual  match
     heart_rate <class 'numpy.float32'> float32  False
burned_calories <class 'numpy.float32'> float32  False
           step <class 'numpy.float32'> float32  False
 step_frequency <class 'numpy.float32'> float32  False
          speed <class 'numpy.float32'> float32  False
        m_speed <class 'numpy.float32'> float32  False
       distance <class 'numpy.float32'> float32  False
     m_activity  <class 'numpy.uint16'> float32  False
     m_ambience  <class 'numpy.uint16'> float32  False
        m_light <class 'numpy.float32'> float32  False
        w_light <class 'numpy.float32'> float32  False
          m_ble  <class 'numpy.uint16'> float32  False
   m_screen_use  <class 'numpy.uint16'> float32  False
  m_usage_stats <class 'numpy.float32'> float32  False
     m_charging  <class 'numpy.uint16'> float32  False

📋 df_lifelog_2_cleaned dtype 점검
         sensor                expected  actual  match

## 날짜별 분리 및 저장

In [ ]:
for i in range(1, 11):
    try:
        df_cleaned = globals()[f"df_lifelog_{i}_cleaned"]
        save_dir = f"data/daily/id{i:02d}"
        os.makedirs(save_dir, exist_ok=True)

        saved_files = []

        for date, group in df_cleaned.groupby('lifelog_date'):
            date_str = pd.to_datetime(date).strftime("%Y-%m-%d")
            path = f"{save_dir}/{date_str}.csv"

            group = group.drop(columns=['lifelog_date'])
            df_t = group.set_index("data").T

            # 센서 순서 정렬
            df_t = df_t[sensor_order]

            # 타입 변환
            for col in df_t.columns:
                if col in mean_sensors:
                    df_t[col] = df_t[col].astype("float32")
                else:
                    df_t[col] = df_t[col].astype("uint16")

            # index 제거
            df_t.reset_index(drop=True, inplace=True)

            df_t.to_csv(path, index=False)
            saved_files.append(path)

        globals()[f"saved_files_{i}"] = saved_files
        print(f"✅ id{i:02d} 저장 완료 ({len(saved_files)}개 파일)")

    except KeyError as e:
        print(f"❌ df_lifelog_{i}_cleaned 없음: {e}")
    except Exception as e:
        print(f"⚠️ 저장 중 오류 (id{i:02d}): {e}")

✅ id01 저장 완료 (81개 파일)
✅ id02 저장 완료 (91개 파일)
✅ id03 저장 완료 (85개 파일)
✅ id04 저장 완료 (91개 파일)
✅ id05 저장 완료 (84개 파일)
✅ id06 저장 완료 (83개 파일)
✅ id07 저장 완료 (85개 파일)
✅ id08 저장 완료 (87개 파일)
✅ id09 저장 완료 (84개 파일)
✅ id10 저장 완료 (84개 파일)


In [ ]:
# 불러올 파일 경로
file_path = "data/daily/id01/2024-06-26.csv"

try:
    df = pd.read_csv(file_path)

    print(f"✅ 파일 로드 성공: {file_path}\n")
    print("📊 컬럼별 데이터 타입:")
    print(df.dtypes)

except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
except PermissionError:
    print(f"❌ 파일에 접근할 수 없습니다 (권한 오류): {file_path}")
except Exception as e:
    print(f"⚠️ 알 수 없는 오류 발생: {e}")

✅ 파일 로드 성공: data/daily/id01/2024-06-26.csv

📊 컬럼별 데이터 타입:
heart_rate         float64
burned_calories    float64
step               float64
step_frequency     float64
speed              float64
m_speed            float64
distance           float64
m_activity           int64
m_ambience           int64
m_light            float64
w_light            float64
m_ble                int64
m_screen_use         int64
m_usage_stats      float64
m_charging           int64
dtype: object
